In [ ]:
import os
#virtually move to parent directory
os.chdir("CLIP-dissect")

import torch

import matplotlib
from matplotlib import pyplot as plt

import utils
import data_utils
import similarity
import pandas as pd

In [ ]:
figure = 'CLIP_layer3'

settings = {
            'CLIP_layer3': {"target_name":'resnet50', "target_layer": 'layer3', "neurons_to_display": "CLIP-Dissect"}
           }

target_name = settings[figure]["target_name"]
target_layer = settings[figure]["target_layer"]
neurons_to_display = settings[figure]["neurons_to_display"]

clip_name = 'ViT-B/16'
d_probe = 'broden'
concept_set = 'data/broden_labels_clean.txt'
batch_size = 200
device = 'cuda'
pool_mode = 'avg'

save_dir = 'saved_activations'
similarity_fn = similarity.soft_wpmi

utils.save_activations(clip_name = clip_name, 
                       target_name = target_name, target_layers = [target_layer], 
                       d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                       device = device, pool_mode=pool_mode, save_dir = save_dir)


save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                  target_layer = target_layer, d_probe = d_probe,
                                  concept_set = concept_set, pool_mode=pool_mode,
                                  save_dir = save_dir)

target_save_name, clip_save_name, text_save_name = save_names

similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                             text_save_name, similarity_fn, device=device)

with open(concept_set, 'r') as f: 
    words = (f.read()).split('\n')

pil_data = data_utils.get_data(d_probe)

top_vals, top_ids = torch.topk(target_feats, k=5, dim=0)

name_conversion = {'resnet50':'resnet50_imagenet', 'resnet18_places':'resnet18_places365'}
ood_names = {'resnet50':'p', 'resnet18_places': 'i'}

# Load Network Dissection results
netdissect_res = pd.read_csv('data/NetDissect_results/{}_{}.csv'.format(name_conversion[target_name],
                                                                       target_layer))
nd_ious = netdissect_res['score'].values
nd_labels = netdissect_res['label'].values

#Load MILAN results
milan_base = pd.read_csv('data/MILAN_results/m_base_{}.csv'.format(name_conversion[target_name]))
if target_name == 'resnet50':
    milan_ood = pd.read_csv('data/MILAN_results/m_places365_{}.csv'.format(name_conversion[target_name]))
elif target_name == 'resnet18_places':
    milan_ood = pd.read_csv('data/MILAN_results/m_imagenet_{}.csv'.format(name_conversion[target_name]))
    
milan_base = milan_base[milan_base['layer']==target_layer]
milan_base = milan_base.sort_values(by=['unit'])
milan_base = list(milan_base['description'])

milan_ood = milan_ood[milan_ood['layer']==target_layer]
milan_ood = milan_ood.sort_values(by=['unit'])
milan_ood = list(milan_ood['description'])

# Calculate which neurons to show
if type(neurons_to_display)==list:
    ids_to_check = neurons_to_display
if neurons_to_display == "CLIP-Dissect":
    ids_to_check = torch.sort(torch.max(similarities, dim=1)[0], descending=True)[1][0:100]
elif neurons_to_display == "NetDissect":
    ids_to_check = torch.sort(torch.tensor(nd_ious), descending=True)[1][0:100]

    
#plot figures
font_size=14
font = {'size'   : font_size}

matplotlib.rc('font', **font)

fig = plt.figure(figsize=[10, len(ids_to_check)*2.2])
subfigs = fig.subfigures(nrows=len(ids_to_check), ncols=1)
for j, orig_id in enumerate(ids_to_check):
    vals, ids = torch.topk(similarities[orig_id], k=5, largest=True)
        
    subfig = subfigs[j]
    subfig.text(0.13, 0.96, "Neuron {}:".format(int(orig_id)), size=font_size)
    subfig.text(0.27, 0.96, "CLIP-Dissect:", size=font_size)
    subfig.text(0.4, 0.96, words[int(ids[0])], size=font_size)
    subfig.text(0.53 ,0.96, "NetDissect:", size=font_size)
    subfig.text(0.65 ,0.96, nd_labels[orig_id], size=font_size)
    
    subfig.text(0.13, 0.85, "MILAN(b):", size=font_size)
    subfig.text(0.24, 0.85, milan_base[orig_id], size=font_size)
    
    subfig.text(0.53, 0.85, "MILAN({}):".format(ood_names[target_name]), size=font_size)
    subfig.text(0.63, 0.85, milan_ood[orig_id], size=font_size)
    axs = subfig.subplots(nrows=1, ncols=5)
    for i, top_id in enumerate(top_ids[:, orig_id]):
        im, label = pil_data[top_id]
        im = im.resize([375,375])
        axs[i].imshow(im)
        axs[i].axis('off')
plt.show()

In [ ]:
neuron_data = []

# Iterate through neurons and gather data
for j, orig_id in enumerate(ids_to_check):
    vals, ids = torch.topk(similarities[orig_id], k=5, largest=True)

    # Gather information for each neuron
    neuron_info = {
        'Neuron_Number': int(orig_id),
        'CLIP-Dissect': words[int(ids[0])],
        'NetDissect': nd_labels[orig_id],
        'MILAN(b)': milan_base[orig_id],
        'MILAN({})'.format(ood_names[target_name]): milan_ood[orig_id]
    }

    neuron_data.append(neuron_info)

# Create a DataFrame
df = pd.DataFrame(neuron_data)

# Save DataFrame to Excel with encoding specified when reading the CSV
excel_file_path = 'neuron_data_layer3_label.xlsx'
df.to_excel(excel_file_path, index=False, engine='openpyxl')

print(f'Data saved to {excel_file_path}')